BioMedClip-PubMedBert Example testing

In [1]:
import numpy as np
import pandas as pd

In [2]:
file_path = "../Datasets/ROCO2/test_images/test/"
# Read the text file into a DataFrame
df = pd.read_csv("../Datasets/ROCO2/test_captions.csv", sep=',', header=1, names=['ID', 'Caption'])
# Replace the ID column with the Image column
df['Image'] = df['ID'] + ".jpg"
# Drop the old ID column
df.drop('ID', axis=1, inplace=True)
df.head()

,Caption,Image
0,Computed tomography (CT) shows floating thromb...,ROCOv2_2023_test_000002.jpg
1,Digitally subtracted angiogram demonstrates ac...,ROCOv2_2023_test_000003.jpg
2,Digitally subtracted angiogram of the IMA demo...,ROCOv2_2023_test_000004.jpg
3,Angle measurement of a Type 1 canal.,ROCOv2_2023_test_000005.jpg
4,Computed tomography on day 26Follow-up enhance...,ROCOv2_2023_test_000006.jpg


In [3]:
import numpy as np
import torch

embeddings_npz = np.load('embeddings/chest-xray-classification/biomedclip-pubmedbert-chuhac_validation.npz')
print(list(embeddings_npz.keys()))

['image_embeddings', 'labels', 'class_embeddings']


In [5]:
image_embeddings = embeddings_npz['image_embeddings']  
class_embeddings = embeddings_npz['class_embeddings']  
labels = embeddings_npz['labels']
# Convert numpy array to torch tensor
embeddings_tensor = torch.tensor(image_embeddings)

# Assuming the model expects input tensors named 'inputs' or similar
inputs = {'inputs': embeddings_tensor}

# Forward pass through the model
outputs = model(**inputs)

NameError: name 'model' is not defined

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, FeatureExtractionPipeline

# Template and labels
template = 'this is a photo of '
labels = [
    'adenocarcinoma histopathology',
    'brain MRI',
    'covid line chart',
    'squamous cell carcinoma histopathology',
    'immunohistochemistry histopathology',
    'bone X-ray',
    'chest X-ray',
    'pie chart',
    'hematoxylin and eosin histopathology'
]

# Load tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained("chuhac/BiomedCLIP-vit-bert-hf")

# Initialize CLIP processor with an image processor
image_processor = FeatureExtractionPipeline(model="clip", framework="pt", device=device)

# Ensure that the model and processor are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("chuhac/BiomedCLIP-vit-bert-hf").to(device)
processor = CLIPProcessor(tokenizer=tokenizer, image_processor=image_processor, device=device)

# Test images
dataset_url = 'https://huggingface.co/microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224/resolve/main/example_data/biomed_image_classification_example_data/'
test_imgs = [
    'squamous_cell_carcinoma_histopathology.jpeg',
    'H_and_E_histopathology.jpg',
    'bone_X-ray.jpg',
    'adenocarcinoma_histopathology.jpg',
    'covid_line_chart.png',
    'IHC_histopathology.jpg',
    'chest_X-ray.jpg',
    'brain_MRI.jpg',
    'pie_chart.png'
]

# Generate texts as a list of strings
texts = [template + l for l in labels]

# Perform zero-shot classification for each image
for img_name in test_imgs:
    img_url = dataset_url + img_name
    
    # Encode the image and texts using the processor
    inputs = processor(text=texts, images=img_url, return_tensors="pt", padding=True)
    
    # Perform forward pass
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Print results
    print(f"Results for {img_name}:")
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)  # Convert logits to probabilities
    for i, (label, prob) in enumerate(zip(labels, probs[0])):
        print(f"{label}: {prob:.4f}")
    print()